### Loading data

In [1]:
import keras
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, Flatten, Dense, Input, Reshape, Dropout, InputLayer
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from keras.models import Sequential, Model
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import cv2
import os
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
np.random.seed(11)

In [2]:
train_dir = os.path.join(os. getcwd() , r"E:\vs code\DeepLearning Task\train")
test_dir = os.path.join (os.getcwd(), r"E:\vs code\DeepLearning Task\test")
categories = ['benign','malignant']
y_train = []
x_train = []
x_test = []
y_test = []
Batch_Size = 32
INIT_LR = 1e-4
EPOCHES = 10

In [3]:
for category in categories:
  path = os.path.join(train_dir, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path)
    image = cv2.resize(image, (224,224))
    image = image / 255
    x_train.append(image)
    y_train.append(category)

In [4]:
for category in categories:
  path = os.path.join(test_dir, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path)
    image - cv2.resize(image, (224,224))
    image = image / 255
    x_test.append(image)
    y_test.append(category)

In [5]:
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical

lb = LabelBinarizer ()
y_train = lb.fit_transform(y_train)
y_train = to_categorical(y_train,num_classes=2)
y_test = lb.fit_transform(y_test)
y_test = to_categorical (y_test, num_classes=2)

In [6]:
x_train = np.array(x_train, dtype= 'float32')
y_train = np.array(y_train,dtype= 'float32')
x_test = np.array(x_test,dtype= 'float32')
y_test = np.array(y_test, dtype= 'float32')

In [7]:
x_train.shape

(2637, 224, 224, 3)

In [8]:
x_test.shape

(660, 224, 224, 3)

## 5) Only CNN

In [9]:
model = Sequential ()
model.add(InputLayer (input_shape= (224, 224, 3)))
model.add(Conv2D (32, kernel_size= (2,2),activation='relu')) 
model.add (MaxPooling2D (pool_size=(2,2))) 
model.add(Conv2D (64, kernel_size= (2,2),activation='relu')) 
model.add(MaxPooling2D (pool_size= (2,2))) 
model.add(Conv2D (128,kernel_size= (2,2),activation='relu')) 
model.add(MaxPooling2D(pool_size= (2,2))) 
model.add(Flatten ()) 

model.add(Dense (128, activation='relu')) 
model.add (Dropout (0.5))
model.add (Dense (2,activation= 'softmax' ))

In [11]:
model.compile (optimizer=Adam(learning_rate=INIT_LR, weight_decay=INIT_LR/Batch_Size),
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [12]:
fittinig = model.fit(x_train, y_train,
                    epochs=EPOCHES,
                    batch_size=Batch_Size)

Epoch 1/10
83/83 [==============================] - 61s 708ms/step - loss: 0.6073 - accuracy: 0.6614
Epoch 2/10
83/83 [==============================] - 61s 732ms/step - loss: 0.4731 - accuracy: 0.7740
Epoch 3/10
83/83 [==============================] - 60s 727ms/step - loss: 0.4497 - accuracy: 0.7865
Epoch 4/10
83/83 [==============================] - 60s 720ms/step - loss: 0.4282 - accuracy: 0.7880
Epoch 5/10
83/83 [==============================] - 61s 735ms/step - loss: 0.4175 - accuracy: 0.7998
Epoch 6/10
83/83 [==============================] - 59s 705ms/step - loss: 0.4149 - accuracy: 0.8047
Epoch 7/10
83/83 [==============================] - 59s 716ms/step - loss: 0.3977 - accuracy: 0.8081
Epoch 8/10
83/83 [==============================] - 61s 729ms/step - loss: 0.4064 - accuracy: 0.8013
Epoch 9/10
83/83 [==============================] - 61s 732ms/step - loss: 0.3985 - accuracy: 0.8146
Epoch 10/10
83/83 [==============================] - 60s 728ms/step - loss: 0.3884 - accura

## 6) CNN and ResNet50

In [13]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [14]:
input_shape = (224, 224, 3)
# Input layer
input_img = Input(shape=input_shape)

### ResNet50

In [16]:
# Load the pre-trained ResNet50 model without top layers and set input
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_img)

# Freeze the weights of the ResNet50 layers (optional)
for layer in base_model.layers:
    layer.trainable = False

### CNN

In [29]:
# Custom CNN layers on top of ResNet50

x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(base_model.output)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
classification_output = Dense(2, activation='softmax')(x)

In [30]:
# Create the classification model
classifier = Model(inputs=input_img, outputs=classification_output)

In [31]:
# Compile the classification model
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
classifier.fit(x_train, y_train,
               epochs=10,
               batch_size=32)

Epoch 1/10
83/83 [==============================] - 135s 2s/step - loss: 0.7129 - accuracy: 0.5142
Epoch 2/10
83/83 [==============================] - 130s 2s/step - loss: 0.6904 - accuracy: 0.5427
Epoch 3/10
83/83 [==============================] - 135s 2s/step - loss: 0.6884 - accuracy: 0.5461
Epoch 4/10
83/83 [==============================] - 139s 2s/step - loss: 0.6856 - accuracy: 0.5472
Epoch 5/10
83/83 [==============================] - 138s 2s/step - loss: 0.6854 - accuracy: 0.5487
Epoch 6/10
83/83 [==============================] - 133s 2s/step - loss: 0.6920 - accuracy: 0.5256
Epoch 7/10
83/83 [==============================] - 125s 2s/step - loss: 0.6900 - accuracy: 0.5461
Epoch 8/10
83/83 [==============================] - 125s 2s/step - loss: 0.6894 - accuracy: 0.5461
Epoch 9/10
83/83 [==============================] - 126s 2s/step - loss: 0.6893 - accuracy: 0.5461
Epoch 10/10
83/83 [==============================] - 126s 2s/step - loss: 0.6893 - accuracy: 0.5461
